<a href="https://colab.research.google.com/github/antonimmo/ebc-jupyternotebooks/blob/master/Spectra_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade netcdf4 geopandas folium

In [0]:
# Monta la nube de GoogleDrive en el sistema "local", e importa algunas librerías necesarias
#from importlib import reload
from google.colab import drive
drive.mount('/gdrive')

In [0]:
prnt = '/gdrive/My Drive/Tesis/spectral_analysis/'
from shutil import copyfile
tools_fn = 'spectral_analysis_tools.py'

In [0]:
import folium
from folium.plugins import MousePosition
import json
import pandas as pd
import geopandas as gpd
import branca

In [5]:
var = 'KE'
season = 'ASO'
data = pd.read_csv('{}/{}_{}_data.csv'.format(prnt,var,season))
data.head()

,Id,avg
0,584,0.242632
1,882,0.142390
2,794,0.390995
3,425,0.296442
4,261,0.534730


In [6]:
#with open('{}/{}_{}_geo.json'.format(prnt,var,season),'r') as json_file:
#    geo_data = json.load(json_file)
#print(geo_data)

geodata = gpd.read_file('{}/{}_{}_geo.json'.format(prnt,var,season),driver='GeoJSON')
geodata = geodata.set_index('s_id',drop=False).join(data.set_index('Id'))
#geodata.sort_values(by=['lat','lon']).head()
geodata.head()

,s_id,var,season,lat,lon,geometry,avg
s_id,,,,,,,
584,584,KE,ASO,-16.398,91.0,"POLYGON ((88 -19.0555, 88 -13.7405, 94 -13.740...",0.242632
882,882,KE,ASO,51.994,-35.0,"POLYGON ((-38 50.24049999999999, -38 53.747500...",0.142390
794,794,KE,ASO,36.055,-29.0,"POLYGON ((-32 33.80649999999999, -32 38.303500...",0.390995
425,425,KE,ASO,-40.411,91.0,"POLYGON ((88 -42.51450000000001, 88 -38.3075, ...",0.296442
261,261,KE,ASO,-55.395,-143.0,"POLYGON ((-146 -57.01250000000001, -146 -53.77...",0.534730


In [7]:
min_v, max_v = geodata['avg'].quantile([0.05,0.95]).apply(lambda x: round(x, 2))
cmap = branca.colormap.LinearColormap(
    colors=['#fcfbfd','#efedf5','#dadaeb','#bcbddc','#9e9ac8','#807dba','#6a51a3','#54278f','#3f007d'],
    #index=geodata['avg'].quantile([0.2,0.4,0.6,0.8]),
    vmin=min_v,
    vmax=max_v,
    caption='Average {}'.format(var)
)

style_fn = lambda x: {
    'fillColor': cmap(x['properties']['avg']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.2,
    'opacity':0.2
}

cmap

In [8]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
m = folium.Map(location=[0,0], zoom_start=3, tiles='Stamen Terrain')

data_layer = folium.GeoJson(
    geodata,
    name=var,
    style_function=style_fn,
    tooltip=folium.GeoJsonTooltip(
        fields=['s_id','lat','lon'],
        aliases=['Id','Lat','Lon'],
        localize=True
    )
).add_to(m)

#folium.Choropleth(
#    #geo_data='{}/{}_{}_geo.json'.format(prnt,var,season),
#    geo_data=geo_data,
#    name='choropleth',
#    data=data,
#    columns=['Id','avg'],
#    key_on='feature.id',
#    fill_color='YlGn',
#    fill_opacity=0.2,
#    line_opacity=0.2,
#    legend_name='{} avg'.format(var),
#    control_scale=True
#).add_to(m)


m.add_child(folium.TileLayer('OpenStreetMap'))
m.add_child(folium.LatLngPopup())
m.add_child(MousePosition())
m.add_child(cmap)
m.add_child(folium.LayerControl())

m